In [1]:
import geopandas as gpd
from sqlalchemy import create_engine
import pandas as pd

print("Библиотеки успешно импортированы!")

Библиотеки успешно импортированы!


In [2]:
# --- Ваши параметры подключения ---
db_host = 'localhost'
db_port = '5433'         # <-- Ваш порт из docker -p
db_user = 'postgres'     # <-- Ваш POSTGRES_USER
db_pass = 'postgres'     # <-- Ваш POSTGRES_PASSWORD
db_name = 'meteorites_db'  # <-- Ваш POSTGRES_DB

# Создаем строку подключения
db_connection_str = f"postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"

# Создаем "движок" подключения
try:
    db_connection = create_engine(db_connection_str)
    print(f"Успешное подключение к базе: {db_name}")
except Exception as e:
    print(f"ОШИБКА ПОДКЛЮЧЕНИЯ: {e}")

Успешное подключение к базе: meteorites_db


In [3]:
# Загружаем полигоны стран
try:
    countries_gdf = gpd.read_postgis(
        'SELECT * FROM countries', 
        db_connection, 
        geom_col='geom'
    )
    print(f"Загружено {len(countries_gdf)} стран.")

    # Загружаем точки метеоритов
    meteorites_gdf = gpd.read_postgis(
        'SELECT * FROM meteorites', 
        db_connection, 
        geom_col='geom'
    )
    print(f"Загружено {len(meteorites_gdf)} метеоритов.")

except Exception as e:
    print(f"ОШИБКА ЗАГРУЗКИ ДАННЫХ: {e}")
    print("Проверьте, что таблицы 'countries' и 'meteorites' существуют в базе 'meteorites_db'")

Загружено 258 стран.
Загружено 45716 метеоритов.


In [5]:
# --- ВАЖНО: Проверка CRS ---
# Системы координат (CRS) ДОЛЖНЫ совпадать для корректной работы.
if countries_gdf.crs != meteorites_gdf.crs:
    print(f"Внимание! CRS не совпадают. Приводим CRS метеоритов ({meteorites_gdf.crs}) к CRS стран ({countries_gdf.crs}).")
    meteorites_gdf = meteorites_gdf.to_crs(countries_gdf.crs)

print("Выполняю пространственное объединение...")

# sjoin = spatial join (пространственное объединение)
# 'inner' - оставляет только те метеориты, которые НАШЛИСЬ внутри страны
# ИСПРАВЛЕНО: используем 'predicate' вместо 'op'
joined_gdf = gpd.sjoin(meteorites_gdf, countries_gdf, how='inner', predicate='intersects')

print("Объединение завершено.")
# joined_gdf теперь содержит метеориты, к которым добавилась информация о стране
# (включая колонку с названием страны)

Выполняю пространственное объединение...
Объединение завершено.


In [7]:
# Посмотрите на доступные колонки:
print(f"Доступные колонки из таблицы стран: {joined_gdf.columns}")

country_name_column = 'name_right'

# Группируем по названию страны и считаем количество (size)
results = joined_gdf.groupby(country_name_column).size()

# Превращаем результат в красивый DataFrame
results_df = results.reset_index(name='meteorite_count')

# Сортируем по убыванию
results_sorted = results_df.sort_values(by='meteorite_count', ascending=False)

# --- Финальный результат ---
print("\nРезультат (Топ 10 стран):")
print(results_sorted.head(10))

Доступные колонки из таблицы стран: Index(['ogc_fid_left', 'name_left', 'id', 'nametype', 'recclass', 'mass (g)',
       'fall', 'year', 'reclat', 'reclong', 'geolocation', 'geom',
       'index_right', 'ogc_fid_right', 'name_right', 'iso3166_1_alpha_3',
       'iso3166_1_alpha_2'],
      dtype='object')

Результат (Топ 10 стран):
                   name_right  meteorite_count
3                  Antarctica            22090
79                       Oman             3094
114  United States of America             1657
61                      Libya             1467
6                   Australia              640
1                     Algeria              621
23                      Chile              403
56                      Kenya              227
69                    Morocco              224
45                      India              133
